In [ ]:
from typing import Dict

from src.base.forecasting.models import TimeSeriesModel
from src.projects.fagradalsfjall.common.project_settings import (
    CV_HORIZON_SAMPLES,
    TS_CV_SPLITTER,
    TS_PRIMARY_METRIC,
)
from src.base.forecasting.models.time_series import (
    TimeSeriesModelNaiveConstant,
    TimeSeriesModelNaiveExponentialDecay,
    TimeSeriesModelNaiveMean,
)
from src.projects.fagradalsfjall.common.dataset import load_train_data_numpy
from src.projects.fagradalsfjall.common.model_eval import ModelEvalResult
from src.projects.fagradalsfjall.common.model_repo import ModelRepo

# 1. Training - Cross-validation - Evaluation

In [ ]:
# -------------------------------------------------------------------------
#  Load TRAINING set
# -------------------------------------------------------------------------
print("Loading training dataset...    ", end="")
x_train = load_train_data_numpy()
print("Done.")

In [ ]:
# -------------------------------------------------------------------------
#  NAIVE MODELS - Define, train & validate
# -------------------------------------------------------------------------

# --- define ----------------------------------------------
naive_models = {
    "naive-constant": TimeSeriesModelNaiveConstant(),
    "naive-mean": TimeSeriesModelNaiveMean(),
}  # type: Dict[str, TimeSeriesModel]

# --- train -----------------------------------------------
for model_name, model in naive_models.items():
    model.fit(x_train)

# --- validate --------------------------------------------
naive_model_eval_results = ModelEvalResult.eval_many(naive_models)

# --- save results ----------------------------------------
ModelRepo.save_models(naive_model_eval_results)

In [ ]:
# -------------------------------------------------------------------------
#  EXPONENTIAL DECAY - Define, tune & validate
# -------------------------------------------------------------------------

# --- define model ------------------------------------
model_exp_decay = TimeSeriesModelNaiveExponentialDecay()
cv_models = {
    "naive-exp-decay": model_exp_decay,
}  # type: Dict[str, TimeSeriesModel]

# --- run cross-validation ----------------------------
param_grid = {"tau": [1, 2, 3, 4, 5, 6, 8, 12, 16, 20, 24, 30, 36, 48, 60, 72, 96, 120, 144]}

model_exp_decay.cv.grid_search(
    x=x_train,
    param_grid=param_grid,
    metric=TS_PRIMARY_METRIC,
    ts_cv_splitter=TS_CV_SPLITTER,
    hor=CV_HORIZON_SAMPLES,
    retrain=True,
)

# --- validate ----------------------------------------
cv_model_eval_results = ModelEvalResult.eval_many(cv_models)

# --- save results ----------------------------------------
ModelRepo.save_models(cv_model_eval_results)